In [1]:
# Import some modules
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os

In [2]:
def take_images():
    # Function to load calibration images (chessboard images)
    route_folder = "C:/Users/USUARIO/OneDrive - Gasoline Grill/Documentos/DTU/Perception for Autonomous Systems/Final project/34759_final_project_rect/34759_final_project_rect/calib"
    
    route_images_left = route_folder + '/image_02/data'
    route_images_right = route_folder + '/image_03/data'

    images_left = glob.glob(route_images_left+'/*.png')
    images_right = glob.glob(route_images_right+'/*.png')
    images_left.sort()
    images_right.sort()
    assert images_left
    assert images_right
    return images_left, images_right

def take_seq_raw_images():
    # Function to load raw images (seq 01)
    route_folder = "C:/Users/USUARIO/OneDrive - Gasoline Grill/Documentos/DTU/Perception for Autonomous Systems/Final project/34759_final_project_raw/34759_final_project_raw/seq_01"
    route_images_left = route_folder + '/image_02/data'
    route_images_right = route_folder + '/image_03/data'

    images_left = glob.glob(route_images_left+'/*.png')
    images_right = glob.glob(route_images_right+'/*.png')
    images_left.sort()
    images_right.sort()
    assert images_left
    assert images_right
    return images_left, images_right

def take_seq_rect_images():
    # Function to load rect images (seq 01)
    route_folder = "C:/Users/USUARIO/OneDrive - Gasoline Grill/Documentos/DTU/Perception for Autonomous Systems/Final project/34759_final_project_rect/34759_final_project_rect/seq_01"
    route_images_left = route_folder + '/image_02/data'
    route_images_right = route_folder + '/image_03/data'

    images_left = glob.glob(route_images_left+'/*.png')
    images_right = glob.glob(route_images_right+'/*.png')
    images_left.sort()
    images_right.sort()
    assert images_left
    assert images_right
    return images_left, images_right

In [3]:
def show_image(img, time):
    # Function to display an image using OpenCV
    cv2.imshow('image',img)
    cv2.waitKey(time)

In [4]:
def detect_chessboard_corners(image, size, flags=None, is_gray=True):
    # Function to detect chessboard corners in an image
    if not is_gray :
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(image, size, flags)
    return ret, corners

In [5]:
def change_size(image, scale):
    # Function to resize an image by a given scale
    w ,h = image.shape[:2]
    imagen_duplicated = cv2.resize(image, (h * scale, w * scale), interpolation=cv2.INTER_LINEAR)
    return imagen_duplicated

In [6]:
def transform_image(image, square_size = (400,400), start = (0,0)):
    # Function to crop an image into a square of a specific size starting from a given point
    h, w = image.shape[:2]
    x_start, y_start = start
    x_end, y_end = min(x_start + square_size[0], h), min(y_start + square_size[1], w)
    new_image = image[x_start:x_end, y_start:y_end]
    return new_image

In [7]:
def take_position_and_start():
    # Function to provide the positions and sizes for image regions to search for chessboards
    starts_l = [(0,0),(80,300),(60,380), (245,470),(385,390),(280,680),(80,790),(280,800),(50,1010),(220,1070),(370,950),(100,1210),(100,1310)]
    starts_r = [(0,0),(130,250),(80,340), (210,400),(380,380),(250,560),(0,620),(250,690),(0,850),(180,940),(350,860),(100,1120),(100,1190)]

    size_l = [(400,350),(250,200),(100,300),(150,200),(120,300),(500,100),(200,150),(500,100),(200,250),(170,170),(250,250),(380,110),(400,105)]
    size_r = [(400,300),(200,200),(100,200),(180,180),(130,200),(200,170),(300,300),(200,160),(250,300),(200,200),(200,200),(400,90),(400,100)]
    
    return starts_l,size_l, starts_r, size_r

In [8]:
def take_squares():
    # Function to return a predefined list of chessboard square numbers
    squares = [(7,11),(7,11),(5,7),(7,5),(5,7),(7,5),(5,7),(7,5),(5,7),(7,5),(5,7),(15,5),(7,5)]
    return squares

In [9]:
def detect_multiple_chessboards(image, which, is_gray=False, flags = None, scale=3, show_images=False):
    # Function to detect multiple chessboards in a single image, for both left or right camera
    if not is_gray:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    start_l, size_l, start_r, size_r = take_position_and_start()
    squares = take_squares()
    founds = []
    detected_corners = []
    found = False

    for i, (stl, str, szl, szr) in enumerate(zip(start_l, start_r, size_l, size_r)):
        new_image = image.copy()
        if which == 'l':
            st,sz = stl, szl
        else:   
            st,sz = str,szr
        sq = squares[i]
        
        new_image = transform_image(new_image, square_size=sz, start=st)
        new_image = change_size(new_image,scale)        
        ret, corners = detect_chessboard_corners(new_image, sq, flags=flags)
        
        if show_images:
            show_image(new_image,0)

        if ret:
            incremento = np.array([st[1],st[0]]).reshape(1, 1, 2)
            div_scale = np.array([scale]).reshape(1,1,1)
            corners2 = (corners/div_scale) + incremento
            corners2 = corners2.astype(corners.dtype)
            corners = corners2.copy()

            if corners[0,0,0] > corners[-1,0,0]:
                corners = np.flip(corners, axis=0)
            corners = np.array(corners, dtype=np.float32)

            detected_corners.append(corners)
            founds.append(True)
            found = True
        else:
            founds.append(False)
            detected_corners.append([])

    return found, detected_corners, founds

In [10]:
def calibrate_camera_for_intrinsic_parameters(images, which, show_images=False, check_images=[]):
    # Function to calibrate the camera by detecting chessboards and estimating intrinsic parameters
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_COUNT + cv2.TERM_CRITERIA_COUNT, 40, 0.001)
    number_of_chessboards = 13
    squares = take_squares()
    world_scaling = 10
    flags =  cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK
    conv_size = (3,3)


    objpoints_dict = {}
    for row,col in squares:
        objp = np.zeros((row*col,3), np.float32)
        objp[:,:2] = np.mgrid[0:row,0:col].T.reshape(-1,2)
        objpoints_dict[(row,col)] = world_scaling * objp

    img = cv2.imread(images[0])
    height,width = img.shape[:2]
        
    imgpoints = [] 
    objpoints = [] 


    for i, frame in enumerate(images):
        if check_images is not None and frame[len(frame)-6:] not in check_images:
            continue

        frame = cv2.imread(frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        ret,corners,founds = detect_multiple_chessboards(gray, which = which, is_gray=True, flags=flags)

        if ret:
            for i in range(number_of_chessboards):
                if founds[i]:
                    corners[i] = cv2.cornerSubPix(gray, corners[i], conv_size, (-1, -1), criteria)
                    objpoints.append(objpoints_dict[squares[i]])
                    imgpoints.append(corners[i])
                    cv2.drawChessboardCorners(frame, squares[i], corners[i], ret)

            if show_images:
                show_image(frame,0)
            cv2.destroyAllWindows()

    cv2.destroyAllWindows()

    ret, cmtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (width, height), None, None)
    print('chessboards detected',sum(founds))
    print('rmse of camera calibration:', ret)
    
    return cmtx, dist

In [11]:
def stereo_calibration(images_left, images_right, cmtx_l, dist_l, cmtx_r, dist_r, show_images=False, check_images=[]):
    # Function to perform stereo calibration using chessboard images
    squares = take_squares()
    world_scaling = 10
    number_of_chessboards = 13
    flags =  cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
    conv_size = (3,3)
    img = cv2.imread(images_left[0])
    height,width = img.shape[:2]

    objpoints = []
    imgpoints_left = []
    imgpoints_right = []

    objpoints_dict = {}
    for size in squares:
        objp = np.zeros((size[0]*size[1],3), np.float32)
        objp[:, :2] = np.mgrid[0:size[0], 0:size[1]].T.reshape(-1, 2)
        objpoints_dict[size] = objp * world_scaling


    for fname_left, fname_right in zip(images_left, images_right):
        if check_images is not None and fname_left[len(fname_left)-6:] not in check_images:
            continue

        img_left = cv2.imread(fname_left)
        img_right = cv2.imread(fname_right)

        gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
        gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

        img_left_draw = img_left.copy()
        img_right_draw = img_right.copy()

        ret_l,corners_l,founds_l = detect_multiple_chessboards(gray_left, which = 'l', is_gray=True, flags=flags)
        ret_r,corners_r,founds_r = detect_multiple_chessboards(gray_right, which = 'r', is_gray=True, flags=flags)
        print('chessboards found', sum(founds_l), sum(founds_r))

        if ret_l and ret_r:

            for i in range(number_of_chessboards):
                if founds_l[i] and founds_r[i]:
                    corners_l[i] = cv2.cornerSubPix(gray_left, corners_l[i], conv_size, (-1, -1), criteria)
                    corners_r[i] = cv2.cornerSubPix(gray_right, corners_r[i], conv_size, (-1, -1), criteria)

                    cl,cr = corners_l[i],corners_r[i] 
                    rows,columns = squares[i]
                    objpoints.append(objpoints_dict[squares[i]])
                    imgpoints_left.append(cl)
                    imgpoints_right.append(cr)

                    cv2.drawChessboardCorners(img_left_draw, (rows,columns), cl, founds_l[i])
                    cv2.drawChessboardCorners(img_right_draw, (rows,columns), cr, founds_r[i])

            if show_images:
                show_image(img_left_draw,0)
                show_image(img_right_draw,0)

        img_left_draw = img_left.copy()
        img_right_draw = img_right.copy()
    

        cv2.destroyAllWindows()

    stereocalibration_flags = 0
    ret, CM1, dist0, CM2, dist1, R, T, E, F = cv2.stereoCalibrate(
            objectPoints=objpoints, 
            imagePoints1=imgpoints_left, 
            imagePoints2=imgpoints_right, 
            cameraMatrix1=cmtx_l, 
            distCoeffs1=dist_l,
            cameraMatrix2=cmtx_r, 
            distCoeffs2=dist_r, 
            imageSize=(width,height), 
            criteria = criteria, 
            flags = stereocalibration_flags)

    print('rmse of stereo calibration: ', ret)

    cv2.destroyAllWindows()
    return ret, CM1, dist0, CM2, dist1, R, T, E, F, objpoints,imgpoints_left, imgpoints_right

In [12]:
def feature_matching_with_ransac(image1, image2):
    # Function to perform feature matching between two images using ORB and RANSAC
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(image1, None)
    kp2, des2 = orb.detectAndCompute(image2, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    _, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    inliers = mask.sum()
    
    return inliers, len(matches)

def fourier_similarity(image1, image2):
    # Function to compute Fourier Transform similarity between two images
    f1 = np.fft.fft2(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY))
    f2 = np.fft.fft2(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY))
    
    f1 = np.fft.fftshift(f1)
    f2 = np.fft.fftshift(f2)
    
    mag1 = np.abs(f1)
    mag2 = np.abs(f2)
    
    similarity = np.sum((mag1 - np.mean(mag1)) * (mag2 - np.mean(mag2)))
    similarity /= np.sqrt(np.sum((mag1 - np.mean(mag1)) ** 2) * np.sum((mag2 - np.mean(mag2)) ** 2))
    return similarity

In [13]:
def stereo_rectification(cmtx_l, dist_l, cmtx_r,dist_r, R_l,R_r,T_l,T_r, compute_similarities = True, show_images=False, undistort=False, check_images = None):
    # Function to perform stereo rectification and optionally compute image similarities
    images_raw_left, images_raw_right = take_seq_raw_images()
    images_rect_left, images_rect_right = take_seq_rect_images()
    
    R_l = np.asarray(R_l, dtype=np.float64)
    R_r = np.asarray(R_r, dtype=np.float64)
    T_l = np.asarray(T_l, dtype=np.float64)
    T_r = np.asarray(T_r, dtype=np.float64)
    cmtx_l = np.asarray(cmtx_l, dtype=np.float64)
    cmtx_r = np.asarray(cmtx_r, dtype=np.float64)
    dist_l = np.asarray(dist_l, dtype=np.float64)
    dist_r = np.asarray(dist_r, dtype=np.float64)

    
    h, w = cv2.imread(images_raw_left[0]).shape[:2]
    image_size = (w, h)

    
    R_l, R_r, P_l, P_r, Q, roi_l,roi_r = cv2.stereoRectify(cmtx_l, dist_l, cmtx_r, dist_r, image_size, R_r, T_r, alpha=0.4)

        
    map1_left, map2_left = cv2.initUndistortRectifyMap(
        cmtx_l, dist_l, R_l, P_l, image_size, cv2.CV_32FC1
    )

    map1_right, map2_right = cv2.initUndistortRectifyMap(
        cmtx_r, dist_r, R_r, P_r, image_size, cv2.CV_32FC1
    )

    four_sim_l = []
    feature_sim_l = []
    four_sim_r = []
    feature_sim_r = []

    output_folder_left = "rectified_left"
    output_folder_right = "rectified_right"
    os.makedirs(output_folder_left, exist_ok=True)
    os.makedirs(output_folder_right, exist_ok=True)

    for fraw_left, fraw_right, frect_left, frect_right in zip(images_raw_left, images_raw_right, images_rect_left, images_rect_right):
        if check_images is not None and fraw_left[len(fraw_left)-7:] not in check_images:
            continue

        left_raw_image = cv2.imread(fraw_left)
        right_raw_image = cv2.imread(fraw_right)
        left_rect_image = cv2.imread(frect_left)
        right_rect_image = cv2.imread(frect_right)

        rectified_left = left_raw_image.copy()
        rectified_right = right_raw_image.copy()

        h_l,w_l = left_raw_image.shape[:2]
        h_r,w_r = right_raw_image.shape[:2]


        rectified_left = cv2.remap(rectified_left, map1_left, map2_left, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
        rectified_right = cv2.remap(rectified_right, map1_right, map2_right, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

        x_l,y_l,w_l,h_l = roi_l
        x_r,y_r,w_r,h_r = roi_r

        rectified_left = rectified_left[y_l:y_l+h_l, x_l:x_l+w_l]
        rectified_right = rectified_right[y_r:y_r+h_r, x_r:x_r+w_r]

        # Save rectified images
        left_filename = os.path.join(output_folder_left, os.path.basename(fraw_left))
        right_filename = os.path.join(output_folder_right, os.path.basename(fraw_right))

        cv2.imwrite(left_filename, rectified_left)
        cv2.imwrite(right_filename, rectified_right)

        width = w_l
        height = h_l
        rectified_left = cv2.resize(rectified_left, (width, height))
        left_rect_image = cv2.resize(left_rect_image, (width, height))
        rectified_right = cv2.resize(rectified_right, (width, height))
        right_rect_image = cv2.resize(right_rect_image, (width, height))


        if show_images:
            right_raw_image = cv2.resize(right_raw_image, (width,height))
            cv2.putText(right_raw_image, "Raw Image", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(rectified_right, "Rectified Image (by us)", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(right_rect_image, "Rectified Image (given from assignment)", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            combined_image = np.vstack((right_raw_image, rectified_right,right_rect_image))
            windowname_1 = "Rectified"
            cv2.namedWindow(windowname_1, cv2.WINDOW_NORMAL) 
            cv2.imshow(windowname_1, combined_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        if compute_similarities:
            similarity_l = fourier_similarity(left_rect_image, rectified_left)
            similarity_r = fourier_similarity(right_rect_image, rectified_right)
            four_sim_l.append(similarity_l)
            four_sim_r.append(similarity_r)

            inliers_l, total_matches_l = feature_matching_with_ransac(left_rect_image, rectified_left)
            inliers_r, total_matches_r = feature_matching_with_ransac(right_rect_image, rectified_right)
            similarity_l = inliers_l / total_matches_l
            similarity_r = inliers_r / total_matches_r
            feature_sim_l.append(similarity_l)
            feature_sim_r.append(similarity_r)
    if compute_similarities:
        print('feature_l', sum(feature_sim_l)/len(feature_sim_l))
        print('feature_r', sum(feature_sim_r)/len(feature_sim_r))
        print('four_l', sum(four_sim_l)/len(four_sim_l))
        print('four_r', sum(four_sim_r)/len(four_sim_r))

    print('saved rectified left images to', output_folder_left)
    print('saved rectified right images to', output_folder_right)

In [14]:
if __name__ == '__main__':
    check_images = ['00.png','01.png','02.png']  # select check_images = None for selecting all of the images. If not, select check_images = ['00.png','01.png','02.png', ...]
    images_left, images_right = take_images()
    print('Starting camera calibration for left camera')
    cmtx_l, dist_l = calibrate_camera_for_intrinsic_parameters(images_left, which='l', check_images=check_images, show_images=False)
    print('Starting camera calibration for right camera')
    cmtx_r, dist_r = calibrate_camera_for_intrinsic_parameters(images_right, which = 'r', check_images=check_images, show_images=False)
    print('Starting stereo calibration')
    ret, CM1, dist0, CM2, dist1, R, T, E, F, objpoints,imgpoints_left, imgpoints_right = stereo_calibration(images_left, images_right, cmtx_l, dist_l, cmtx_r, dist_r, check_images=check_images, show_images=False)
    R_l = np.eye(3, dtype=np.float32)
    T_l = np.array([0., 0., 0.]).reshape((3, 1))
    R_r = R
    T_r = T
    print('Starting stereo rectification')
    check_images = None #['000.png', '001.png']   # select check_images = None for selecting all of the images. If not, select check_images = ['000.png','001.png','002.png', ...]
    stereo_rectification(cmtx_l,dist_l,cmtx_r,dist_r,R_l,R_r,T_l,T_r, compute_similarities=False, show_images=False, undistort=True, check_images = check_images) 

Starting camera calibration for left camera


chessboards detected 12
rmse of camera calibration: 0.22227990020175312
Starting camera calibration for right camera
chessboards detected 11
rmse of camera calibration: 0.1833643034120145
Starting stereo calibration
chessboards found 12 10
chessboards found 11 10
chessboards found 12 11
rmse of stereo calibration:  0.18616252363650612
Starting stereo rectification
saved rectified left images to rectified_left
saved rectified right images to rectified_right
